In [1]:
# %pip install -U llama-index-readers-file pymupdf
# %pip install -U llama-index-core
# %pip install -U llama-index-llms-azure-openai
# %pip install -U llama-index-embeddings-azure-openai
# %pip install -U llama-index-vector-stores-chroma
# %pip install -U python-dotenv

## 1: Data Loading

In [2]:
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader
from llama_index.core import Document

DATA_DIR = Path("data")
DATA_DIR.mkdir(parents=True, exist_ok=True)

loader = PyMuPDFReader()
docs0 = loader.load(file_path=Path("./data/GMDSS_System-IOM_Manual.pdf"))

# Stitch pages together
doc_text = "\n\n".join([d.get_content() for d in docs0])
docs = [Document(text=doc_text)]

## 2: Enhanced Hierarchical Parsing with Summaries

In [3]:
from llama_index.core.node_parser import HierarchicalNodeParser, get_leaf_nodes
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import Settings
from dotenv import load_dotenv
import os

load_dotenv()

# Azure OpenAI Setup
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")
AZURE_CHAT_DEPLOYMENT = os.getenv("AZURE_CHAT_DEPLOYMENT")
AZURE_EMBEDDING_DEPLOYMENT = os.getenv("AZURE_EMBEDDING_DEPLOYMENT")
EMBEDDING_DIMENSIONS = int(os.getenv("EMBEDDING_DIMENSIONS", 3072))

# Initialize LLM (use gpt-4o-mini for cost savings on summaries)
llm = AzureOpenAI(
    model="gpt-4o",
    deployment_name=AZURE_CHAT_DEPLOYMENT,
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=OPENAI_API_VERSION,
    temperature=0.1,
)

embed_model = AzureOpenAIEmbedding(
    model="text-embedding-3-large",
    deployment_name=AZURE_EMBEDDING_DEPLOYMENT,
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=OPENAI_API_VERSION,
    dimensions=EMBEDDING_DIMENSIONS,
)

Settings.embed_model = embed_model
Settings.llm = llm

## Generate Parent Summaries

In [4]:
def generate_parent_summaries(nodes):
    """
    Generate concise summaries for parent nodes
    
    Args:
        nodes: List of all nodes from HierarchicalNodeParser
        use_mini: Use gpt-4o-mini for cost savings (recommended)
    """
    from llama_index.core.schema import NodeRelationship
    
    summaries = {}
    parent_nodes = [n for n in nodes if NodeRelationship.CHILD in n.relationships]
    
    summary_llm = llm
    
    print(f"Generating summaries for {len(parent_nodes)} parent nodes...")
    print(f"Using: {summary_llm.model}")
    
    for i, node in enumerate(parent_nodes):
        if i % 10 == 0:
            print(f"  Progress: {i}/{len(parent_nodes)}")
        
        prompt = f"""Provide a concise summary (2-3 sentences, max 100 tokens) of this text section:

{node.get_content()[:3000]}

Summary:"""
        
        try:
            response = summary_llm.complete(prompt)
            summaries[node.node_id] = response.text.strip()
        except Exception as e:
            print(f"Warning: Failed to generate summary for node {node.node_id}: {e}")
            summaries[node.node_id] = node.get_content()[:150] + "..."
    
    print(f"✓ Generated {len(summaries)} summaries")
    return summaries

## Enrich Leaf Nodes with Hierarchy Context

In [5]:
def enrich_leaf_nodes_with_context(nodes, parent_summaries):
    """
    Add parent context to leaf nodes for better retrieval
    
    Returns: List of enriched TextNode objects
    """
    from llama_index.core.schema import TextNode, NodeRelationship
    
    leaf_nodes = get_leaf_nodes(nodes)
    enriched_nodes = []
    
    print(f"Enriching {len(leaf_nodes)} leaf nodes with parent context...")
    
    for leaf in leaf_nodes:
        # Build hierarchy chain
        hierarchy_chain = []
        current = leaf
        
        # Traverse up the parent chain
        while NodeRelationship.PARENT in current.relationships:
            parent_id = current.relationships[NodeRelationship.PARENT].node_id
            parent_node = next((n for n in nodes if n.node_id == parent_id), None)
            
            if parent_node and parent_node.node_id in parent_summaries:
                hierarchy_chain.insert(0, parent_summaries[parent_node.node_id])
            
            current = parent_node
            if current is None:
                break
        
        # Create enriched content with context breadcrumbs
        if hierarchy_chain:
            context_str = " → ".join(hierarchy_chain)
            enriched_content = f"[CONTEXT: {context_str}]\n\n{leaf.get_content()}"
        else:
            enriched_content = leaf.get_content()
        
        # Create new enriched node
        enriched_node = TextNode(
            text=enriched_content,
            metadata={
                **leaf.metadata,
                "hierarchy_depth": len(hierarchy_chain),
                "has_context": len(hierarchy_chain) > 0,
                "original_node_id": leaf.node_id
            },
            relationships=leaf.relationships
        )
        enriched_node.node_id = leaf.node_id
        
        enriched_nodes.append(enriched_node)
    
    print(f"✓ Enriched {len(enriched_nodes)} nodes")
    return enriched_nodes

## Create Nodes with Enhanced Context

In [ ]:
# Parse with your existing settings
node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[4096, 1024, 512]  # Your current setup
)
nodes = node_parser.get_nodes_from_documents(docs)

print(f"Total nodes created: {len(nodes)}")
print(f"Leaf nodes: {len(get_leaf_nodes(nodes))}")

# ENHANCEMENT: Generate summaries and enrich leaf nodes
parent_summaries = generate_parent_summaries(nodes)
enriched_leaf_nodes = enrich_leaf_nodes_with_context(nodes, parent_summaries)

Total nodes created: 1219
Leaf nodes: 934
Generating summaries for 285 parent nodes...
Using: gpt-4o
  Progress: 0/285


2026-01-16 12:57:35,644 - INFO - HTTP Request: POST https://nilup-mgaf895d-eastus.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2026-01-16 12:57:37,328 - INFO - HTTP Request: POST https://nilup-mgaf895d-eastus.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2026-01-16 12:57:39,164 - INFO - HTTP Request: POST https://nilup-mgaf895d-eastus.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2026-01-16 12:57:41,592 - INFO - HTTP Request: POST https://nilup-mgaf895d-eastus.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2026-01-16 12:57:43,471 - INFO - HTTP Request: POST https://nilup-mgaf895d-eastus.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.

  Progress: 10/285


2026-01-16 12:57:56,693 - INFO - HTTP Request: POST https://nilup-mgaf895d-eastus.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2026-01-16 12:57:58,419 - INFO - HTTP Request: POST https://nilup-mgaf895d-eastus.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2026-01-16 12:57:59,959 - INFO - HTTP Request: POST https://nilup-mgaf895d-eastus.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2026-01-16 12:58:02,002 - INFO - HTTP Request: POST https://nilup-mgaf895d-eastus.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2026-01-16 12:58:04,254 - INFO - HTTP Request: POST https://nilup-mgaf895d-eastus.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.

  Progress: 20/285


...

  Progress: 270/285


2026-01-16 13:07:28,612 - INFO - HTTP Request: POST https://nilup-mgaf895d-eastus.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2026-01-16 13:07:30,734 - INFO - HTTP Request: POST https://nilup-mgaf895d-eastus.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2026-01-16 13:07:32,688 - INFO - HTTP Request: POST https://nilup-mgaf895d-eastus.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2026-01-16 13:07:34,932 - INFO - HTTP Request: POST https://nilup-mgaf895d-eastus.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2026-01-16 13:07:37,178 - INFO - HTTP Request: POST https://nilup-mgaf895d-eastus.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.

  Progress: 280/285


2026-01-16 13:07:48,706 - INFO - HTTP Request: POST https://nilup-mgaf895d-eastus.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2026-01-16 13:07:51,621 - INFO - HTTP Request: POST https://nilup-mgaf895d-eastus.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2026-01-16 13:07:53,470 - INFO - HTTP Request: POST https://nilup-mgaf895d-eastus.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2026-01-16 13:07:55,302 - INFO - HTTP Request: POST https://nilup-mgaf895d-eastus.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2026-01-16 13:07:57,046 - INFO - HTTP Request: POST https://nilup-mgaf895d-eastus.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.

✓ Generated 285 summaries
Enriching 934 leaf nodes with parent context...
✓ Enriched 934 nodes


## 4: Storage and Indexing (ChromaDB Option)

In [8]:
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb


docstore = SimpleDocumentStore()
docstore.add_documents(nodes)  # Store ALL nodes for AutoMergingRetriever

chroma_client = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = chroma_client.get_or_create_collection("gmdss_manual")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

storage_context_chroma = StorageContext.from_defaults(
    vector_store=vector_store,
    docstore=docstore
)

base_index = VectorStoreIndex(
    enriched_leaf_nodes,
    storage_context=storage_context_chroma,
)

2026-01-16 14:07:08,369 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2026-01-16 14:07:11,147 - INFO - HTTP Request: POST https://nilup-mgaf895d-eastus.cognitiveservices.azure.com/openai/deployments/text-embedding-3-large/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2026-01-16 14:07:13,192 - INFO - HTTP Request: POST https://nilup-mgaf895d-eastus.cognitiveservices.azure.com/openai/deployments/text-embedding-3-large/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2026-01-16 14:07:14,975 - INFO - HTTP Request: POST https://nilup-mgaf895d-eastus.cognitiveservices.azure.com/openai/deployments/text-embedding-3-large/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2026-01-16 14:07:16,756 - INFO - HTTP Request: POST https://nilup-mgaf895d-eastus.cognitiveservices.azure.com/openai/deployments/text-embedding-3-large/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2026-0

## Querying

In [9]:
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

base_retriever = base_index.as_retriever(similarity_top_k=6)
retriever = AutoMergingRetriever(
    base_retriever, 
    storage_context_chroma, 
    verbose=True
)

query_engine = RetrieverQueryEngine.from_args(retriever)

# Test query
query_str = "when Adjusting the squelch level what do When hearing noise or an unwanted signal?"
response = query_engine.query(query_str)

print("\n" + "="*80)
print("QUERY RESPONSE")
print("="*80)
print(str(response))

2026-01-16 14:16:16,267 - INFO - HTTP Request: POST https://nilup-mgaf895d-eastus.cognitiveservices.azure.com/openai/deployments/text-embedding-3-large/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2026-01-16 14:16:16,496 - INFO - > Merging 4 nodes into parent node.
> Parent node id: a29e8cb8-819a-4680-b52f-01d0ae189856.
> Parent node text: LT-3100S GMDSS User & Installation Manual Rev. 1.06
System Submenus
Lars Thrane A/S
www.thrane.eu...



> Merging 4 nodes into parent node.
> Parent node id: a29e8cb8-819a-4680-b52f-01d0ae189856.
> Parent node text: LT-3100S GMDSS User & Installation Manual Rev. 1.06
System Submenus
Lars Thrane A/S
www.thrane.eu...



2026-01-16 14:16:20,019 - INFO - HTTP Request: POST https://nilup-mgaf895d-eastus.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"



QUERY RESPONSE
When hearing noise or an unwanted signal, briefly push the Volume/Squelch button to ensure the squelch bar is visible on the display. Then, turn the squelch button clockwise until the radio is muted, effectively suppressing the noise.


## Cost Tracking
